# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import nltk
import re
# Import word_tokenize and stopwords from nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

import nltk
import nltk.corpus
from gensim.models.word2vec import Word2Vec
from sklearn.pipeline import Pipeline


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate

from sklearn.metrics import confusion_matrix, f1_score, classification_report

In [2]:
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential

In [3]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

# Import dataset

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = pd.read_csv("/content/drive/My Drive/nela10sample.csv")
data.content=data.content.astype(str)
data.title=data.title.astype(str)

In [6]:
data

,id,date,source,title,content,author,url,published,published_utc,collection_utc,Reliability
0,abcnews--2020-04-15--Students could take SAT a...,2020-04-15,abcnews,Students could take SAT at home if schools rem...,A home version of the SAT college entrance exa...,NaN,https://abcnews.go.com/US/wireStory/students-s...,"Wed, 15 Apr 2020 19:33:47 -0400",1586993627,1586993864,0
1,abcnews--2020-04-15--Detroit facing fresh fisc...,2020-04-15,abcnews,Detroit facing fresh fiscal woes with coronavi...,DETROIT -- Barely surviving the 2008 economic ...,NaN,https://abcnews.go.com/US/wireStory/detroit-fa...,"Wed, 15 Apr 2020 19:30:18 -0400",1586993418,1586993865,0
2,abcnews--2020-04-15--Thousands protest Michiga...,2020-04-15,abcnews,Thousands protest Michigan governor's social d...,"LANSING, Mich. -- Thousands of flag-waving, ho...",NaN,https://abcnews.go.com/US/wireStory/hundreds-p...,"Wed, 15 Apr 2020 18:48:35 -0400",1586990915,1586993866,0
3,abcnews--2020-04-15--Baltimore Police defends ...,2020-04-15,abcnews,Baltimore Police defends planned aerial survei...,The Baltimore Police Department on Wednesday d...,NaN,https://abcnews.go.com/US/wireStory/baltimore-...,"Wed, 15 Apr 2020 18:32:47 -0400",1586989967,1586993867,0
4,abcnews--2020-04-15--Attorney who fought for c...,2020-04-15,abcnews,Attorney who fought for civil rights in Louisi...,NEW ORLEANS -- A white lawyer who defended bla...,NaN,https://abcnews.go.com/US/wireStory/attorney-f...,"Wed, 15 Apr 2020 18:12:18 -0400",1586988738,1586993864,0
...,...,...,...,...,...,...,...,...,...,...,...
102549,wearechange--2020-04-26--North Korea Power Vac...,2020-04-26,wearechange,North Korea Power Vacuum Brings US And China T...,The fall of North Korea could lead to a US war...,AJ Nelson,https://wearechange.org/north-korea-power-vacu...,"Sun, 26 Apr 2020 20:12:36 +0000",1587946356,1587951770,2
102550,wearechange--2020-04-26--Twitter Suspends Acco...,2020-04-26,wearechange,Twitter Suspends Account of Biotech Company Te...,Twitter has suspended the account of a Colorad...,AJ Nelson,https://wearechange.org/twitter-suspends-accou...,"Sun, 26 Apr 2020 19:14:56 +0000",1587942896,1587951772,2
102551,wearechange--2020-04-26--Riots Have Started In...,2020-04-26,wearechange,Riots Have Started In Venezuela! Will It Come ...,"OH YEAH, since we are not corporate or governm...",AJ Nelson,https://wearechange.org/riots-have-started-in-...,"Sun, 26 Apr 2020 17:48:15 +0000",1587937695,1587951770,2
102552,wearechange--2020-04-27--True Coronavirus Deat...,2020-04-27,wearechange,True Coronavirus Death Toll Likely 60% Higher ...,As experts try to model the true number of cor...,AJ Nelson,https://wearechange.org/true-coronavirus-death...,"Mon, 27 Apr 2020 02:10:52 +0000",1587967852,1587991692,2


In [7]:
# Concetanate title and content
data["content"] = data["title"] + " " + data["content"]

#Keep only content, Reliability as columns
data = data.drop(['id','date','source','title','author','url','published','published_utc','collection_utc'],axis=1)

In [8]:
data

,content,Reliability
0,Students could take SAT at home if schools rem...,0
1,Detroit facing fresh fiscal woes with coronavi...,0
2,Thousands protest Michigan governor's social d...,0
3,Baltimore Police defends planned aerial survei...,0
4,Attorney who fought for civil rights in Louisi...,0
...,...,...
102549,North Korea Power Vacuum Brings US And China T...,2
102550,Twitter Suspends Account of Biotech Company Te...,2
102551,Riots Have Started In Venezuela! Will It Come ...,2
102552,True Coronavirus Death Toll Likely 60% Higher ...,2


In [ ]:
print (data.content[0])

Students could take SAT at home if schools remain closed A home version of the SAT college entrance exam is being prepared in case schools remain closed into the fall, College Board officials said Wednesday as they announced the cancellation of June testing.

Instead of a paper-and-pencil test given under proctors' supervision, the home version would be digital and rely on “remote proctoring.” That could include using the computer’s camera and microphone to monitor movement or talking, College Board President Jeremy Singer said on a conference call with reporters.

The rival ACT also will launch an at-home option in late fall or early winter, the exam's administrators said Wednesday.

“We would much prefer that schools reopen but we are ready to innovate and deliver in the unlikely case we need to,” College Board Chief Executive David Coleman said.

Coronavirus-related school closures forced the cancellation of spring testing for about 1 million first-time SAT test-takers, the majority

In [9]:
def process_statements(text):
    
    # Tokenize the words
    tokenized = word_tokenize(text)

    # Remove the stop words
    tokenized = [token for token in tokenized if token not in stopwords.words("english")] 

    # Lemmatize the words, changing text to lowercase
    lemmatizer = WordNetLemmatizer()
    tokenized = [lemmatizer.lemmatize(token.lower(), pos='a') for token in tokenized]

    # Remove non-alphabetic characters and keep the words contains three or more letters
    tokenized = [token for token in tokenized if token.isalpha() and len(token)>2]
    
    return tokenized
    
# Call the function and store the result into a new column
#train["Processed"] = train['Statement'].str.lower().apply(process_statements)

# Print the first fifteen rows of Processed
#display(train[["Processed"]].head(15))

In [10]:
data.head

<bound method NDFrame.head of                                                   content  Reliability
0       Students could take SAT at home if schools rem...            0
1       Detroit facing fresh fiscal woes with coronavi...            0
2       Thousands protest Michigan governor's social d...            0
3       Baltimore Police defends planned aerial survei...            0
4       Attorney who fought for civil rights in Louisi...            0
...                                                   ...          ...
102549  North Korea Power Vacuum Brings US And China T...            2
102550  Twitter Suspends Account of Biotech Company Te...            2
102551  Riots Have Started In Venezuela! Will It Come ...            2
102552  True Coronavirus Death Toll Likely 60% Higher ...            2
102553  WHO Mysteriously Deletes Tweet About Reinfecti...            2

[102554 rows x 2 columns]>

In [14]:
df1 = data[:5000]
df2 = data[-5000:]
df3 = [df1,df2]
smallerdf = pd.concat(df3)

In [15]:
smallerdf.shape

(10000, 2)

In [16]:
smallerdf.head

<bound method NDFrame.head of                                                   content  Reliability
0       Students could take SAT at home if schools rem...            0
1       Detroit facing fresh fiscal woes with coronavi...            0
2       Thousands protest Michigan governor's social d...            0
3       Baltimore Police defends planned aerial survei...            0
4       Attorney who fought for civil rights in Louisi...            0
...                                                   ...          ...
102549  North Korea Power Vacuum Brings US And China T...            2
102550  Twitter Suspends Account of Biotech Company Te...            2
102551  Riots Have Started In Venezuela! Will It Come ...            2
102552  True Coronavirus Death Toll Likely 60% Higher ...            2
102553  WHO Mysteriously Deletes Tweet About Reinfecti...            2

[10000 rows x 2 columns]>

In [17]:
# split the dataset into training and testing datasets 

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(smallerdf['content'], smallerdf['Reliability'], test_size=0.20)

In [18]:
print (X_train)

98494     Tragic Wycombe fan dies of heart attack after ...
97752     Man Utd fans plan mass Old Trafford walk-out d...
100664    Nashville Authorities Arrest Convicted Felon f...
102456    Authorities charge 10 in alleged MS-13 deaths,...
2553      'Threat' - Aston Villa youngster delights as m...
                                ...                        
4083      Meet the billionaires bankrolling Trump's 2020...
100663    Report: FBI Director Wray Behind Suppression o...
1490      Grieving Kenyan grandmother calls for better r...
2473      Former Birmingham City star trolls Newcastle U...
4555      A US doctor predicts a 'massive surge' in coro...
Name: content, Length: 8000, dtype: object


In [19]:
# Apply text pre-processing, then vectorize the text so the models can actually interpret it
vectorizer = TfidfVectorizer(analyzer=process_statements)
x_train_vectorized = vectorizer.fit_transform(X_train)
x_test_vectorized = vectorizer.transform(X_test)

In [20]:
print (x_train_vectorized.shape)
print (x_test_vectorized.shape)
print (y_train.shape)
print (y_test.shape)

(8000, 60311)
(2000, 60311)
(8000,)
(2000,)


#Logistic Regression

In [21]:
log_reg = LogisticRegression()
log_reg.fit(x_train_vectorized, y_train)
logR_predicted = log_reg.predict(x_test_vectorized)
print(f"{log_reg.__class__.__name__} accuracy: {log_reg.score(x_test_vectorized, y_test)}")

LogisticRegression accuracy: 0.8365


In [22]:
print(classification_report(y_test, logR_predicted))
print('\n')
print(confusion_matrix(y_test, logR_predicted))

              precision    recall  f1-score   support

           0       0.83      0.86      0.85      1037
           2       0.85      0.81      0.83       963

    accuracy                           0.84      2000
   macro avg       0.84      0.84      0.84      2000
weighted avg       0.84      0.84      0.84      2000



[[896 141]
 [186 777]]


In [26]:
cv_results = cross_validate(log_reg, x_train_vectorized, y_train, cv=5)
cv_results['test_score'].mean()

0.8355

#Random Forest

In [23]:
random_forest = RandomForestClassifier(n_estimators=10, max_depth=100)
random_forest.fit(x_train_vectorized, y_train)

print(f"{random_forest.__class__.__name__} accuracy: {random_forest.score(x_test_vectorized, y_test)}")

RandomForestClassifier accuracy: 0.75


In [24]:
rForest_pred = random_forest.predict(x_test_vectorized)
print(classification_report(y_test, rForest_pred))
print('\n')
print(confusion_matrix(y_test, rForest_pred))

              precision    recall  f1-score   support

           0       0.75      0.78      0.76      1037
           2       0.75      0.72      0.74       963

    accuracy                           0.75      2000
   macro avg       0.75      0.75      0.75      2000
weighted avg       0.75      0.75      0.75      2000



[[804 233]
 [267 696]]


In [25]:
cv_results = cross_validate(random_forest, x_train_vectorized, y_train, cv=5)
cv_results['test_score'].mean()

0.74325

# Multinomial Naive Bayes

In [29]:
multi_nb = MultinomialNB()
multi_nb.fit(x_train_vectorized, y_train)
print(f"{multi_nb.__class__.__name__} accuracy: {multi_nb.score(x_test_vectorized, y_test)}")

MultinomialNB accuracy: 0.7805


In [30]:
NaiveB_pred = multi_nb.predict(x_test_vectorized)
print(classification_report(y_test, NaiveB_pred))
print('\n')
print(confusion_matrix(y_test, NaiveB_pred))

              precision    recall  f1-score   support

           0       0.78      0.81      0.79      1037
           2       0.78      0.75      0.77       963

    accuracy                           0.78      2000
   macro avg       0.78      0.78      0.78      2000
weighted avg       0.78      0.78      0.78      2000



[[838 199]
 [240 723]]


In [31]:
cv_results = cross_validate(multi_nb, x_train_vectorized, y_train, cv=5)
cv_results['test_score'].mean()

0.7823749999999999

#SVM

In [32]:
svm = svm.LinearSVC()
svm.fit(x_train_vectorized, y_train)
print(f"{svm.__class__.__name__} accuracy: {svm.score(x_test_vectorized, y_test)}")

LinearSVC accuracy: 0.8585


In [33]:
svm_pred = svm.predict(x_test_vectorized)

print(classification_report(y_test, svm_pred))
print('\n')
print(confusion_matrix(y_test, svm_pred))

              precision    recall  f1-score   support

           0       0.85      0.88      0.87      1037
           2       0.87      0.83      0.85       963

    accuracy                           0.86      2000
   macro avg       0.86      0.86      0.86      2000
weighted avg       0.86      0.86      0.86      2000



[[916 121]
 [162 801]]


In [34]:
cv_results = cross_validate(svm, x_train_vectorized, y_train, cv=5)
cv_results['test_score'].mean()

0.8480000000000001

# LSTM with GloVe embedding

In [ ]:
!pip install glove_python

In [ ]:
lemmatizer = WordNetLemmatizer()
stops = set(stopwords.words('english'))
# Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stops:
            final_text.append(i.strip())
    return " ".join(final_text)

# Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
    
# Removing the noisy text
def denoise_text(text):
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text


# Apply function on statement column
statement_train = [denoise_text(s) for s in train['statement'].values]
statement_test = [denoise_text(s) for s in test['statement'].values]
statement_valid = [denoise_text(s) for s in valid['statement'].values]

# Get lables (Consider combine the levels)
label_train = train['label'].values
label_test = test['label'].values
label_valid = valid['label'].values

In [ ]:
table = str.maketrans('', '', string.punctuation)

def makeWords(sentences):
  wordList = []
  for headline in sentences:
    words = headline.split(' ')
    stripped = [w.strip().translate(table) for w in words]
    wordList.append(stripped)
  return wordList

words_train = makeWords(statement_train)
words_test = makeWords(statement_test)
words_valid = makeWords(statement_valid)

In [ ]:
MAX_WORDS = 100000 # max number of words for tokenizer
MAX_SEQUENCE_LENGTH = 1000 # max length of each sentences, including padding
EMBEDDING_DIM = 100 # embedding dimensions for word vectors

In [ ]:
#X_rain to sequences

# prepare tokenizer
tokenizer = Tokenizer(num_words=MAX_WORDS)

#preparing vocabulary
tokenizer.fit_on_texts(words_train)

#converting text into integer sequences
sequences = tokenizer.texts_to_sequences(words_train) 

#padding to prepare sequences of same length
X_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')

# record the size of the vocab
vocab_size=len(tokenizer.word_index) + 1 #+1 for padding
print(vocab_size)

In [ ]:
# X_test to sequences

tokenizer = Tokenizer(num_words=MAX_WORDS)

#preparing vocabulary
tokenizer.fit_on_texts(words_test)

#converting text into integer sequences
sequences = tokenizer.texts_to_sequences(words_test) 

#padding to prepare sequences of same length
X_test = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding = 'post')

#Glove_Embedding Using Gensim

In [ ]:
from __future__ import print_function
import argparse
import pprint
import gensim

from glove import Glove
from glove import Corpus

In [ ]:
# Build the corpus dictionary and the cooccurrence matrix.
print('Pre-processing corpus')

corpus_model = Corpus()
corpus_model.fit(words_train, window=10)
#Corpus.save('corpus.model')
        
print('Dict size: %s' % len(corpus_model.dictionary))
print('Collocations: %s' % corpus_model.matrix.nnz)

In [ ]:
# Fit glove vectorizer with concurrence matrix from Corpus class
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus_model.matrix)
glove.add_dictionary(corpus_model.dictionary)

In [ ]:
# Pre-trained embedding

EMBEDDING_FILE = ("/content/drive/My Drive/glove.6B.100d.txt")

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open(EMBEDDING_FILE)

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

In [ ]:
# create a weight matrix for words
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = embeddings_index[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

# Defining Neural Network

In [ ]:
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
#LSTM 
model.add(LSTM(units=128))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

del embedding_matrix

In [ ]:
history = model.fit(X_train, y_train, epochs=3, validation_data=(X_valid, y_valid))

In [ ]:
y_pred_glove = (model.predict(X_test) > 0.5).astype("int")

In [ ]:
print("Accuracy is {} for LSTM with GloVe embedding.".format(accuracy_score(y_test, y_pred_glove)))

In [ ]:
print(classification_report(y_test, y_pred, zero_division = 1))

In [ ]:
#plot accuracy and loss
def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")